In [1]:
from IPython.core.magic import register_line_cell_magic
from IPython.display import display, Math

@register_line_cell_magic
def hy(line, cell=None):
    "Simple cell magic to execute Hy code within a Python kernel."
    import hy; code = (line or "") + (cell or "")
    return hy.eval(hy.read_str("(do\n" + code + "\n)\n"), globals())
del hy

def print_math(math):
    "Prints formatted LaTeX math notation."
    display(Math(math))

In [2]:
%%hy

(require [lemma.core :as le])
(import [lemma.exceptions [LeEquationError]])
(import [lemma.algebra [seq-sum pow length add sub PI
                        div/frac :as div
                        mul/times :as mul]])

## Simple expression

In [3]:
%%hy

;; Compile a lemma expression with le.expr
(setv simple-expression
 (le.expr (add 1 PI)))

simple-expression

LeExpression#'(LeOperator#add 1 LeConstant#PI)

In [4]:
%%hy

;; Generate latex for a lemma expression 
(print-math (.latex simple-expression))

<IPython.core.display.Math object>

In [5]:
%%hy

;; Execute a lemma expression like a function
(simple-expression)

4.141592653589793

## More complex example (with `seq-sum` and an identifier)

In [6]:
%%hy

;; Specify latex formatter for variable names.
(le.def-identifier beta r"\beta")

(setv complex-expression
 (le.expr
  (seq-sum [a [1 2]
            beta [3 4]]
   (sub 1 (div a beta)))))

(print-math (.latex complex-expression))
(print (complex-expression))

<IPython.core.display.Math object>

2.25


In [7]:
%%hy

(import [hy.contrib.hy-repr [hy-repr]])

;; You can even print the Hy code for an expression.
(print (hy-repr (.hy complex-expression)))

'(sum (lfor a [1 2] beta [3 4] (do (require hy.contrib.walk) (hy.contrib.walk.let [args [1 (do (require hy.contrib.walk) (hy.contrib.walk.let [numerator a denominator beta] (do (/ numerator denominator))))]] (do (hy.core.shadow.- #* args))))))


## Order of operations

In [8]:
%%hy

;; Parens will be automatically added based on operator precedence.
(setv order-examples
 [(le.expr (sub (add 1 2) 3))
  (le.expr (sub 1 (add 2 3)))
  (le.expr (sub 1 (div 2 3)))])

(for [example order-examples]
  (print-math (.latex example))
  (print (example))
  (print "-----"))

<IPython.core.display.Math object>

0
-----


<IPython.core.display.Math object>

-4
-----


<IPython.core.display.Math object>

0.33333333333333337
-----


In [9]:
%%hy

;; Use #p to explicitly add parens.
(setv parens-example (le.expr (sub 1 #p(div 2 3))))
(print-math (.latex parens-example))
(print (parens-example))

<IPython.core.display.Math object>

0.33333333333333337


In [10]:
%%hy

;; Use #b to explicitly remove parens.
(setv bare-example (le.expr (add 1 #b(add 2 3))))
(print-math (.latex bare-example))
(print (bare-example))

<IPython.core.display.Math object>

6


## Formulas

In [11]:
%%hy

(le.def-identifier xs "X")
(le.def-identifier mu r"\mu")

(le.def-formula variance r"\sigma^2"
  [xs mu n]
  "Formula for population variance."
  (div
   (seq-sum [x xs]
    (pow (sub x mu) 2))
   n))

In [12]:
%%hy

;; Use a formula like any other operator
(setv var-expression (le.expr (variance [1 2 3] :mu 2 :n 3)))
(print-math (.latex var-expression))
(print (var-expression))

<IPython.core.display.Math object>

0.6666666666666666


In [13]:
%%hy

;; Or use a formula as an operator whose latex representation is the formula's body
(setv var-expression (le.expr (variance.op [1 2 3] 2 3)))
(print-math (.latex var-expression))
(print (var-expression))

<IPython.core.display.Math object>

0.6666666666666666


In [14]:
%%hy

;; Get the signature of a formula
(print-math (.signature-latex variance))

<IPython.core.display.Math object>

In [15]:
%%hy

;; Get the definition of a formula
(print-math (.latex variance))
(print-math (.latex variance [1 2 3] 2 3))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
%%hy

;; Directly run a formula like a function
(variance :xs [0 5 10] :mu 5 :n 3)

16.666666666666668

In [17]:
%%hy

;; Get docstring.
variance.--doc--

'Formula for population variance.'

## Python Interop

In [18]:
# Use expressions and formulae directly from Python:

print_math(variance.signature_latex())
print_math(variance.latex())
print(variance(xs=[0, 5, 10], mu=5, n=3))
print_math(var_expression.latex())
print(var_expression())

<IPython.core.display.Math object>

<IPython.core.display.Math object>

16.666666666666668


<IPython.core.display.Math object>

0.6666666666666666


## Equations

In [19]:
%%hy

(le.def-identifier z r"\zeta")

;; Equations are defined as a series of equivalent expressions.
;; Useful for working out equation solutions with executable code and then printing out notation.
(le.def-equation my-equation
                 [x &optional [z 4]]
                 "Equation involving FOIL expansion."
                 [(add #b(mul (add x 1) (sub x 1)) z)
                  (add (sub #b(add #b(sub (pow x 2) x) x) 1) z)
                  (add (sub (pow x 2) 1) z)])

(print-math (.latex my-equation))
(print-math (.latex my-equation :x 5))
(my-equation :x 5)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

28

In [20]:
%%hy

my-equation.--doc--

'Equation involving FOIL expansion.'

In [21]:
%%hy

(le.def-equation bad-equation [x]
                 [(add x 1)
                  (sub x 1)])

(print-math (.latex bad-equation))

;; When executing an equation, all expressions are executed,
;; and an exception is raised if any of the results were not equal.
(try
 (bad-equation :x 5)
 (except [ex LeEquationError]
  (print ex)))

<IPython.core.display.Math object>

While evaluating LeEquation#bad-equation with arguments [x=5]: result '4' of LeExpression#'(LeOperator#sub 5 1) did not equal result '6' of LeExpression#'(LeOperator#add 5 1)


## Unit Testing

In [22]:
%%hy

(assert (= ((le.expr (add 1 1)) 2)))
(assert (= (variance [1 2 3] 2 3) (/ 2 3)))

In [23]:
assert var_expression() == 2 / 3
assert variance([1, 2, 3], 2, 3) == 2 / 3

## Extending Lemma

In [24]:
%%hy

;; Define your own constants.
(import math)
(le.def-constant e "e" math.e)

(print-math (.latex e))
(print (e))

<IPython.core.display.Math object>

2.718281828459045


In [25]:
%%hy

;; Define operators using lemma expressions.
(le.def-operator decrement [val]
  (expr (sub val 1)))

(setv dec-expr (le.expr (decrement 3)))
(print-math (.latex dec-expr))
(print (dec-expr))

<IPython.core.display.Math object>

2


In [26]:
%%hy

;; Define operators using Hy expressions.
(le.def-operator increment [val]
  (precedence 100)
  (latex f"{val} + 1")
  (hy (+ val 1)))

(setv inc-expr (le.expr (increment 3)))
(print-math (.latex inc-expr))
(print (inc-expr))

<IPython.core.display.Math object>

4


In [27]:
%%hy

(import [lemma.lang [gen-latex gen-hy]])

;; Define operators using Hy macros (arguments are pass-by-name, and
;; hy-macro is expected to return a quoted Hy expression). Useful for
;; more complex operators that don't just take lemma expressions as
;; arguments (like seq-sum).
(le.def-operator plus2 [val]
  (precedence 100)
  (latex-macro
   f"{(gen-latex val)} + 2")
  (hy-macro
   `(+ ~(gen-hy val) 2)))

(setv plus2-expr (le.expr (plus2 3)))
(print-math (.latex plus2-expr))
(print (plus2-expr))

<IPython.core.display.Math object>

5


In [28]:
%%hy

;; Functions can accept optional args with default values (&rest and &kwargs also work).
(le.def-operator mean
  [vals &optional [val x]]
  (expr (seq-sum [val vals] (div val (length vals)))))

(le.def-identifier g r"\gamma")

(setv mean-example (le.expr (add (mean [1 2 3]) (mean [4 5 6] y) (mean [7 8 9] g))))
(print-math (.latex mean-example))
(print (mean-example))

<IPython.core.display.Math object>

15.0


## Next Steps

* [API reference](https://ben-denham.github.io/lemma/#/lemma.core)
* [Tutorial on adding your own notation to Lemma](https://mybinder.org/v2/gh/ben-denham/lemma/master?filepath=notebooks%2BExtendingTutorial.ipynb)
* [Documentation home](https://ben-denham.github.io/lemma)